Byte-Pair Encoding
==================
'Neural Machine Translation of Rare Words with Subword Units' by Rico Sennrich, Barry Haddow, Alexandria Burch 논문 참고
<br>
<br>
### BPE 적용
OpenNMT 에서는 BPE 적용에 glossary set, vocabulary set 등을 함께 이용한다. glossary set은 task-specific하게 적용하는 등 부가적인 요소이며, vocabulary set은 보통 training data의 word dictionary를 이용해 만든다. 'Attention is all you need!' 논문의 경우 40,000개의 BPE set과 같은 크기의 vocab set을 이용했으며, 같은 계열의 최신 논문인 'Improving Language Understanding by Generative Pre-Training' 역시 같은 confguration을 사용했다.
<br>
<br>
BPE을 새로운 데이터셋에 적용하는 과정을 자세하게 설명해보려 한다.

### BPE 적용 코드 설명
OpenNMT의 apply_bpe는 ①segment -> ②_isolate_glossaries -> ③isolate_glossaries -> ④encode -> ⑤get_pairs -> ⑥check_vocab_and_split -> ⑦recursive_split 의 함수 호출 과정을 거치며 input sentence를 tokenize한다. 

In [13]:
    def segment(self, sentence):
        """segment single sentence (whitespace-tokenized string) with BPE encoding"""
        output = []
        for word in sentence.split():
            new_word = [out for segment in self._isolate_glossaries(word)
                        for out in encode(segment,
                                          self.bpe_codes,
                                          self.bpe_codes_reverse,
                                          self.vocab,
                                          self.separator,
                                          self.version,
                                          self.cache,
                                          self.glossaries)]

            for item in new_word[:-1]:
                output.append(item + self.separator)
            output.append(new_word[-1])

        return ' '.join(output)

__segment(sentence)__<br>
_ex: sentence= "긴 하루였다" -> "긴 하루@@ 였@ 다"_<br>
_input: sentence; whitespace-토큰화된 스트링_<br>
_output: BPE 로직에 의해 분해된 요소들에는 separator가 요소 뒤에 붙은 상태에서 1 whitespace로 구분되는 하나의 스트링_<br>
<br>
예시에서 볼 수 있듯이 원 문장에서 떨어져 있던 단어 사이에는 새로운 구분자가 없다.segment 함수는 BPE를 수행하는 전체과정을 포함하는 함수이다.<br>

In [14]:
    def _isolate_glossaries(self, word):

        word_segments = [word]
        for gloss in self.glossaries:
            word_segments = [out_segments for segment in word_segments
                             for out_segments in isolate_glossary(segment, gloss)]
        return word_segments

___isolate_glossaries(word)__<br>
_ex: glossaries = [땅콩, 비행], word = "비행기땅콩먹는비행기땅콩비행기?" -> ["비행", "기", "땅콩", "먹는", "비행", "기", "땅콩", "비행", "기?"]_<br>
_input: word; 단어 스트링_<br>
_output: glossaries 리스트에 있는 모든 glossary 들을 기준으로 split된 segment 리스트_<br>

In [15]:
def isolate_glossary(word, glossary):
    """
    Isolate a glossary present inside a word.
    Returns a list of subwords. In which all 'glossary' glossaries are isolated 
    For example, if 'USA' is the glossary and '1934USABUSA' the word, the return value is:
        ['1934', 'USA', 'B', 'USA']
    """
    if word == glossary or glossary not in word:
        return [word]
    else:
        splits = word.split(glossary)
        segments = [segment.strip() for split in splits[:-1]
                    for segment in [split, glossary] if segment != '']
        return segments + [splits[-1].strip()] if splits[-1] != '' else segments

__isolate_glossaries(word, glossary)__<br>
_ex: glossary = 땅콩, segment = "땅콩버터먹고땅콩먹자" -> ["땅콩", "버터먹고", "땅콩", "먹자"]_<br>
_input: segment, glossary; 스트링, 스트링_<br>
_output: segments; glossary를 기준으로 split된 segment 리스트_<br>

glossary 스트링이 segment 스트링에 포함된 경우, glossary를 기준으로 split을 수행한다

In [16]:
def encode(orig, bpe_codes, bpe_codes_reverse, vocab, separator, version, cache, glossaries=None):
    """Encode word based on list of BPE merge operations, which are applied consecutively
    """

    # encode는 BPE를 적용하는 함수다. glossary가 주어진 경우, glossary를 기준으로 분해된 segment를 인풋으로 받아 다음의 과정을 수행한다.
    # 1) segment 스트링을 튜플로 분해한 후, get_pairs 함수를 통해 해당 segment 캐릭터들의 pair집합을 구한다
    # 2) pair집합과 bpe_codes 간에 교집합이 없을 때 까지 다음을 수행한다
    # 2-1) pair 묶음들과 bpe_codes의 겹치는 원소 중 가장 처음 원소(빈도수가 가장 많은)를 bigram에 할당
    # 2-2) 1의 튜플을 bigram에 해당하는 스트링의 원소가 있는 인덱스 까지의 원소들을 하나의 스트링으로 만들어 new_word 리스트에 넣고, bigram을 append한 후,  다음 인덱스 부터 2-2를 다시 반복한다
    # 2-3) 2-2에서 구해진 new_word 리스트를 튜플로 변환하고 get_pairs를 통해 pair 집합을 구한다.
    # 3) Vocab가 주어졌다면, 2의 과정을 통해 Byte-pair로 뭉쳐진 토큰들을 check_vocab_and_split을 통해 해당 토큰들이 vocab 집합에 속해있는지 확인 후 OOV들은 잘게 쪼갠다
    # 4) Vocabulary와 잘게 쪼개진 OOV 스트링으로 이루어진 리스트를 반환한다

    if orig in cache:
        return cache[orig]

    if orig in glossaries:
        cache[orig] = (orig,)
        return (orig,)

    word = tuple(orig[:-1]) + (orig[-1] + '</w>',)
    
    pairs = get_pairs(word)

    if not pairs:
        return orig

    while True:
        bigram = min(pairs, key=lambda pair: bpe_codes.get(pair, float('inf')))
        if bigram not in bpe_codes:
            break
        first, second = bigram
        new_word = []
        i = 0
        while i < len(word):
            try:
                j = word.index(first, i)
                new_word.extend(word[i:j])
                i = j
            except:
                new_word.extend(word[i:])
                break

            if word[i] == first and i < len(word) - 1 and word[i + 1] == second:
                new_word.append(first + second)
                i += 2
            else:
                new_word.append(word[i])
                i += 1
        new_word = tuple(new_word)
        word = new_word
        if len(word) == 1:
            break
        else:
            pairs = get_pairs(word)

    # don't print end-of-word symbols
    if word[-1] == '</w>':
        word = word[:-1]
    elif word[-1].endswith('</w>'):
        word = word[:-1] + (word[-1].replace('</w>', ''),)

    if vocab:
        word = check_vocab_and_split(word, bpe_codes_reverse, vocab, separator)

    cache[orig] = word
    return word

encode는 BPE를 적용하는 함수다. glossary가 주어진 경우, glossary를 기준으로 분해된 segment를 인풋으로 받아 다음의 과정을 수행한다.<br>
1. segment 스트링을 튜플로 분해한 후, get_pairs 함수를 통해 해당 segment 캐릭터들의 pair집합을 구한다
2. pair집합과 bpe_codes 간에 교집합이 없을 때 까지 다음을 수행한다
    1. pair 묶음들과 bpe_codes의 겹치는 원소 중 가장 처음 원소(빈도수가 가장 많은)를 bigram에 할당
    2. 1의 튜플을 bigram에 해당하는 스트링의 원소가 있는 인덱스 까지의 원소들을 하나의 스트링으로 만들어 new_word 리스트에 넣고, bigram을 append한 후,  다음 인덱스 부터 B를 다시 반복한다
    3. B에서 구해진 new_word 리스트를 튜플로 변환하고 get_pairs를 통해 pair 집합을 구한다.
3. Vocab가 주어졌다면, 2의 과정을 통해 Byte-pair로 뭉쳐진 토큰들을 check_vocab_and_split을 통해 해당 토큰들이 vocab 집합에 속해있는지 확인 후 OOV들은 잘게 쪼갠다
4. Vocabulary와 잘게 쪼개진 OOV 스트링으로 이루어진 리스트를 반환한다



In [17]:
def get_pairs(word):
    """Return set of symbol pairs in a word.
    word is represented as tuple of symbols (symbols being variable-length strings)
    """
    pairs = set()
    prev_char = word[0]
    for char in word[1:]:
        pairs.add((prev_char, char))
        prev_char = char
    return pairs

__get_pairs(word)__<br>
_ex: word = ["20, "018", "년", "가", "즈"] -> {("20", "018"), ("018", "년"), ("년", "가"), ("가", "즈")}_<br>
_input: word; 스트링 리스트_<br>
_output: pair 집합_<br>
<br>
인풋 리스트 내의 인접한 두 스트링의 pair를 원소로 하는 집합 구하기



In [18]:
def check_vocab_and_split(orig, bpe_codes, vocab, separator):
    """Check for each segment in word if it is in-vocabulary,
    and segment OOV segments into smaller units by reversing the BPE merge operations"""

    # orig; 튜플

    # orig 튜플 속의 원소들이 vocab 집합에 속했는지 확인 후 vocab에 속할 때 까지 recursive_split을 통해 잘게 쪼개 OOV(Out of Vcoabulary)라면 잘게 쪼갠다.
    
    out = []

    for segment in orig[:-1]:
        if segment + separator in vocab:
            out.append(segment)
        else:
            #sys.stderr.write('OOV: {0}\n'.format(segment))
            for item in recursive_split(segment, bpe_codes, vocab, separator, False):
                out.append(item)

    segment = orig[-1]
    if segment in vocab:
        out.append(segment)
    else:
        #sys.stderr.write('OOV: {0}\n'.format(segment))
        for item in recursive_split(segment, bpe_codes, vocab, separator, True):
            out.append(item)

    return out

__check_vocab_and_split(orig, bpe_codes, vocab, separator)__<br>
_input: orig; 튜플_<br>
_output: Vocabulary와 잘게 쪼개진 OOV 스트링으로 이루어진 리스트_<br>
<br>
orig 튜플 속의 원소들이 vocab 집합에 속했는지 확인 후 vocab에 속할 때 까지 recursive_split을 통해 잘게 쪼개 OOV(Out of Vcoabulary)라면 잘게 쪼갠다.

In [19]:
def recursive_split(segment, bpe_codes, vocab, separator, final=False):
    """Recursively split segment into smaller units (by reversing BPE merges)
    until all units are either in-vocabulary, or cannot be split futher."""

    # segment; 스트링

    # 조건부에 일치하는 스트링을 yield로 반환
    # ex. bpe_codes= {'는다</w>': ('는', '다</w>')}, vocab= {'는', '다'}, segment= '먹는다', 
    #     output = '먹는다'
    try:
        if final:
            left, right = bpe_codes[segment + '</w>']
            right = right[:-4]
        else:
            left, right = bpe_codes[segment]
    except:
        #sys.stderr.write('cannot split {0} further.\n'.format(segment))
        yield segment
        return

    if left + separator in vocab:
        yield left
    else:
        for item in recursive_split(left, bpe_codes, vocab, separator, False):
            yield item

    if (final and right in vocab) or (not final and right + separator in vocab):
        yield right
    else:
        for item in recursive_split(right, bpe_codes, vocab, separator, final):
            yield item

__recursive_split(segment, bpe_codes, vocab, separator, final=False)__<br>
_ex: bpecodes= {'는다</w>': ('는', '다</w>')}, vocab= {'는', '다'}, segment= '먹는다'-> '먹는다' (bpe 집합의 key에 매칭되지 않는 토큰의 경우_<br>
_ex: bpecodes= {'는다</w>': ('는', '다</w>')}, vocab= {'는', '다</w>'}, segment= '는다'-> '는', '다</w>'_<br>
_input: segment; 스트링_<br>
_output: 조건부에 일치하는 스트링을 yield로 반환_<br>
<br>
recursive_split은 인풋 segment에 BPE 오퍼레이션을 역으로 수행해 점점 분해해가며 vocab 집합에 속해 있거나, 더 잘게 쪼갤 수 없을 때 까지 반복하는 함수이다. bpe 집합의 key와 비교해 bpe에 있는 토큰이라면 bpe 오퍼레이션을 역으로 수행해 분해한다. 그 후, 역 bpe에 의해 분해된 토큰이 vocab 집합에 속한 원소인지 확인 후 해당된다면 반환하고, 원소가 아니라면 recursive_split을 반복적으로 수행하는 재귀함수.

